# Regional Analyses
Building on the exploratory data analyses, this notebook analyzes and visualizes tree cover statistics at the regional level.

In [ ]:
import os
import rasterio as rs
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd 
import pandas as pd
import matplotlib
import glob

import sys
sys.path.append('scripts/')
import statistical_analyses as st

%load_ext autoreload
%autoreload 2

## Aggregate Regional Stats
Combines country stats spreadsheets into one csv for the region.

In [ ]:
st.create_regional_csv(['Belize', 'El Salvador', 'Honduras', 'Nicaragua', 'Panama', 'Guatemala', 'Costa Rica'], 
                    'central_am')

In [ ]:
st.create_regional_csv(['Burkina Faso', 'Central African Republic','Cameroon', 'Chad', 'Ethiopia','Eritrea',
                    'Mali', 'Mauritania', 'Niger', 'Nigeria', 'Senegal', 'South Sudan', 'Sudan'], 'sahel')

In [ ]:
st.create_regional_csv(['Alagoas', 'Tocantins', 'Roraima', 'Goias', 'Rio de Janeiro'], 'Brazil')

In [ ]:
df = pd.read_csv('statistics/Brazil_statistics_full.csv')
df

## Agricultural and Urban Areas Meeting Forest Cover Definition (>10%)
The FAO defines *forest cover* as a land area of more than 0.5 ha, with a tree canopy cover of more than 10%, which is not primarily under agricultural or other specific non-forest land use (FAO 2000). The stacked barchart illustrates the tree cover in agriculture and urban areas that qualifies as forest under this definition. The light green portion indicates the total percentage of land that is not compliant (<10%) and the dark green indicates the total percentage of land that is compliant (>10%) for each country in a region. All results are for the year 2020.

In [ ]:
def fao_compliant(region, filepath, lcc):
    
    df = pd.read_csv(filepath)
    
    ag_ids = [10.0, 11.0, 12.0, 20.0, 30.0, 40.0]
    urban_ids = [190.0]
    
    if lcc == 'ag':
        df = df[df.esa_id.isin(ag_ids)]
    
    if lcc == 'urban':
        df = df[df.esa_id.isin(urban_ids)]
        
    # get total ha per threshold per country
    df = df.groupby(by=['country', 'tree_cover_class']).sum().reset_index()
    df = df[['country', 'tree_cover_class', 'tof_ha']] 
    
    # remove tree cover classes <10%
    over10 = df[df.tree_cover_class != '0-9']
    over20 = df[(df.tree_cover_class != '0-9') & (df.tree_cover_class != '10-19')]
    over30 = df[(df.tree_cover_class != '0-9') & (df.tree_cover_class != '10-19') & (df.tree_cover_class != '20-29')]
    
    # calculate totals
    total = sum(df.tof_ha)
    over10_ha = sum(over10.tof_ha)
    over10_perc = sum(over10.tof_ha) / total * 100
    over20_perc = sum(over20.tof_ha) / total * 100
    over30_perc = sum(over30.tof_ha) / total * 100
    
    if lcc == 'ag':
        print(f'Total percentage of ag land in {region} that meets FAO forest cover criterion: {round((over10_perc),2)}%')
        print(f'Total percentage of ag land in {region} with >20% tree cover: {round((over20_perc),2)}%')
        print(f'Total percentage of ag land in {region} with >30% tree cover:  {round((over30_perc),2)}%')
        print(f'Total hectares of ag land in {region} that meet FAO forest cover criterion: {over10_ha} ha')
    
    if lcc == 'urban':
        print(f'Total percentage of urban land in {region} that meets FAO forest cover criterion: {round((over10_perc),2)}%')
        print(f'Total percentage of urban land in {region} with >20% tree cover: {round((over20_perc),2)}%')
        print(f'Total percentage of urban land in {region} with >30% tree cover:  {round((over30_perc),2)}%')
        print(f'Total hectares of urban land in {region} that meet FAO forest cover criterion: {over10_ha} ha')

    return None

In [ ]:
fao_compliant('Central America', 'statistics/central_am_statistics_full.csv', 'ag')

In [ ]:
fao_compliant('Malawi', 'statistics/Malawi_statistics_full.csv', 'ag')

In [ ]:
fao_compliant('Kenya', 'Kenya_statistics.csv', 'urban')

In [ ]:
fao_compliant('Sahel', 'statistics/sahel.csv', 'ag')

In [ ]:
def forest_cover_compliant_comb(region, figsize):
    
    region_df = pd.read_csv(f'statistics/{region}.csv')
    
    countries = list(set(region_df.country.values))

    x_labels, bar1, bar2 = [], [], []

    for country in countries:
        
        # filter to one country and only urban/ag land cover classes
        country_df = region_df[region_df.country == country]
        ids = [10.0, 11.0, 12.0, 20.0, 30.0, 40.0, 190.0]
        country_df = country_df[country_df.esa_id.isin(ids)]
        
        # get total tof ha per tree cover threshold
        country_df = country_df.groupby(by=['country', 'tree_cover_class']).sum().reset_index() 
        country_df = country_df[['country', 'tree_cover_class', 'tof_ha']]
        
        # calculate # ha <10% tree cover and >10% tree cover
        under10 = country_df.tof_ha[0]
        over10 = sum(country_df.tof_ha[1:])
        
        # normalize
        total = under10 + over10
        under10_norm = under10/total*100
        over10_norm = over10/total*100
        
        x_labels.append(country)
        bar1.append(over10_norm)
        bar2.append(under10_norm)
   
    # convert to array in order to add data labels
    bar1 = np.asarray(bar1)
    bar2 = np.asarray(bar2)
    
    # create the figure
    plt.figure(figsize=figsize)
    plt.bar(x_labels, bar1, color="seagreen", label='<10% cover')  
    plt.bar(x_labels, bar2, bottom=bar1, color="honeydew", label='>10% cover')    
    
    # labels
    for xpos, ypos, yval in zip(x_labels, bar1/2, bar1):
        plt.text(xpos, ypos, f'{round(yval)}%', ha="center", va="center")
    for xpos, ypos, yval in zip(x_labels, bar1 + bar2/2, bar2):
        plt.text(xpos, ypos, f'{round(yval)}%', ha="center", va="center")
    
    plt.xlabel(' ')
    plt.ylabel('% Land')
    plt.ticklabel_format(style='plain', axis='y')
    plt.title(f'% Agricultural and Urban Land Meeting \n Forest Cover Definition')
    plt.legend(loc='lower right');

In [ ]:
forest_cover_compliant_comb('central_am', (8,6))

In [ ]:
def forest_cover_compliant(region, figsize):
    
    '''
    Legend for ESA land cover categories:
        10: 'Cropland, rainfed',
        11: 'Cropland, rainfed',
        12: 'Cropland, rainfed',
        20: 'Cropland, irrigated or post-flooding',
        30: 'Mosaic cropland / natural vegetation',
        40: 'Mosaic natural vegetation / cropland',
        190: 'Urban areas'
    '''
    
    region_df = pd.read_csv(f'statistics/{region}.csv')
    
    countries = list(set(region_df.country.values))

    x_labels, ag_bar1, ag_bar2, urban_bar1, urban_bar2 = [],[],[],[],[]
    
    for country in countries:
        
        # filter to one country and only urban/ag land cover classes
        country_df = region_df[region_df.country == country]
        ag_ids = [10.0, 11.0, 12.0, 20.0, 30.0, 40.0]
        urban_ids = [190.0]
        ag_df = country_df[country_df.esa_id.isin(ag_ids)].sort_values('country')
        urban_df = country_df[country_df.esa_id.isin(urban_ids)].sort_values('country')
        
        # get total tof ha per tree cover threshold
        ag_df = ag_df.groupby(by=['country', 'tree_cover_class']).sum().reset_index() 
        ag_df = ag_df[['country', 'tree_cover_class', 'tof_ha']] 
        urban_df = urban_df.groupby(by=['country', 'tree_cover_class']).sum().reset_index() 
        urban_df = urban_df[['country', 'tree_cover_class', 'tof_ha']] 
        
        # calculate # ha <10% tree cover and >10% tree cover
        ag_under10 = ag_df.tof_ha[0]
        ag_over10 = sum(ag_df.tof_ha[1:])
        urban_under10 = urban_df.tof_ha[0]
        urban_over10 = sum(urban_df.tof_ha[1:])
        
        # normalize
        ag_under10_norm = ag_under10/(ag_under10 + ag_over10)*100
        ag_over10_norm = ag_over10/(ag_under10 + ag_over10)*100
        urban_under10_norm = urban_under10/(urban_under10 + urban_over10)*100
        urban_over10_norm = urban_over10/(urban_under10 + urban_over10)*100
        
        x_labels.append(country)
        ag_bar1.append(ag_over10_norm) # >10% is on the bottom
        ag_bar2.append(ag_under10_norm) # <10% is on the top
        urban_bar1.append(urban_over10_norm) 
        urban_bar2.append(urban_under10_norm) 
   
    # convert to array in order to add data labels
    ag_bar1 = np.asarray(ag_bar1)
    ag_bar2 = np.asarray(ag_bar2)
    urban_bar1 = np.asarray(urban_bar1)
    urban_bar2 = np.asarray(urban_bar2)

    plt.figure(figsize=figsize)
    
    # create ag error bars
    ag_error = []
    lower = [-2.41, -3.64, -3.99, -3.17, -3.89, -3.93, -3.74]
    upper = [1.05, 0.41, -0.06, 0.53, 0.27, 0.03, 0.19]
    for x,y in zip(lower, upper):
        ag_error.append(y-x)
    
    # Ag plot
    plt.subplot(1,2,1)
    plt.bar(x_labels, ag_bar1, color="seagreen", label='>10% tree cover', yerr=ag_error, capsize=4)  
    plt.bar(x_labels, ag_bar2, bottom=ag_bar1, color="honeydew", label='<10% tree cover')    
    
    # labels
    for xpos, ypos, yval in zip(x_labels, ag_bar1/2, ag_bar1):
        plt.text(xpos, ypos, f'{round(yval)}%', ha="center", va="center")
    for xpos, ypos, yval in zip(x_labels, ag_bar1 + ag_bar2/2, ag_bar2):
        plt.text(xpos, ypos, f'{round(yval)}%', ha="center", va="center")
    
    plt.xlabel(' ')
    plt.ylabel('% Land')
    plt.ticklabel_format(style='plain', axis='y')
    plt.title(f'% Agricultural Land Meeting \n 10% Forest Cover Criteria')
    plt.legend(loc='lower right')
    
    # create urban error bars
    urb_error = []
    lower = [-2.90, -1.91, -3.16, -1.90, -1.94, -2.40, -2.06]
    upper = [1.97, 0.64, 0.17, 0.64, 0.95, 0.92, 0.84]
    for x,y in zip(lower, upper):
        urb_error.append(y-x)

    # Urban plot
    plt.subplot(1,2,2)
    plt.bar(x_labels, urban_bar1, color="seagreen", label='>10% tree cover', yerr=urb_error, capsize=4)  
    plt.bar(x_labels, urban_bar2, bottom=urban_bar1, color="honeydew", label='<10% tree cover')    
    
    # labels
    for xpos, ypos, yval in zip(x_labels, urban_bar1/2, urban_bar1):
        plt.text(xpos, ypos, f'{round(yval)}%', ha="center", va="center")
    for xpos, ypos, yval in zip(x_labels, urban_bar1 + urban_bar2/2, urban_bar2):
        plt.text(xpos, ypos, f'{round(yval)}%', ha="center", va="center")
    
    
    plt.xlabel(' ')
    plt.ylabel('% Land')
    plt.ticklabel_format(style='plain', axis='y')
    plt.title(f'% Urban Land Meeting \n 10% Forest Cover Criteria')
    plt.legend(loc='lower right')
    plt.tight_layout(); 

In [ ]:
forest_cover_compliant('central_am', (14,7))

## Top 5 Admins with Trees on Farms
This function identifies the top five administrative districts with the highest tree cover estimates (average tree cover or total ha) on agriculture land cover types.  

Outstanding items to address:
- If ha of tree cover is the metric, should be weighted

In [ ]:
def top5_admins_ag(filepath):
    
    '''
    Filters dataframe to the following ESA agriculture land cover types:
    
    Cropland, rainfed
    Cropland, rainfed, herbaceous cover
    Cropland, irrigated or post-flooding
    Mosaic cropland / natural vegetation
    Mosaic natural vegetation / cropland 
    
    Calculates the total ha tree cover per admin AND the avg % tree cover per admin.
    Merges the dataframes to include % of land cover class sampled and then builds
    two separate dataframes of top 5 cities in the region with highest stats.
    
    '''
    
    df = pd.read_csv(filepath)
    
    # filter to only ag land cover classes
    ag_ids = [10.0, 11.0, 12.0, 20.0, 30.0, 40.0]
    df = df[df.esa_id.isin(ag_ids)]

    # get total (sum) ha tree cover in ag for each admin (drop ha <10% cover)
    df_ha = df.drop(df[df.tree_cover_class == '0-9'].index)
    df_ha = df.groupby(by=['country', 'admin']).sum().reset_index() 
    df_ha = df_ha[['country', 'admin', 'esa_sampled_ha', 'esa_total_ha','tof_ha']] 
    
    # add column with % of land cover class that was sampled
    df_ha['esa_perc_samp'] = round((df_ha.esa_sampled_ha / df_ha.esa_total_ha) * 100, 2)

    # now use original dataframe to get average tree cover in ag for each admin
    df_avg = df.groupby(by=['country','admin']).mean().reset_index()
    df_avg = df_avg[['country', 'admin', 'tof_mean']]
    
    # merge dataframes
    df_merged = pd.merge(df_ha, df_avg, how='inner', left_on=['country','admin'], right_on=['country','admin'])
    
    # sort and print top 5 ag areas
    top_5_ha = df_merged.sort_values(['tof_ha', 'esa_perc_samp'], ascending=(False, False))[:5]
    top_5_avg = df_merged.sort_values(['tof_mean', 'esa_perc_samp'], ascending=(False, False))[:5]
    
    print('Top 5 Admins with Trees on Farms (total hectares tree cover)')
    for i in top_5_ha.index:
        print(f'Farms in {top_5_ha.admin[i]}, {top_5_ha.country[i]} have {top_5_ha.tof_ha[i]} ' \
              f'ha of tree cover >10%. ({top_5_ha.esa_perc_samp[i]}% of farms were sampled)')
    
    print(' ')
    print('Top 5 Admins with Trees on Farms (average % tree cover)')
    for i in top_5_avg.index:
        print(f'Farms in {top_5_avg.admin[i]}, {top_5_avg.country[i]} have {round(top_5_avg.tof_mean[i], 0)}% ' \
              f'average tree cover. ({top_5_avg.esa_perc_samp[i]}% of farms were sampled)')

    return df_merged

In [ ]:
central_am = top5_admins_ag('statistics/central_america_full.csv')

In [ ]:
sahel = top5_admins_ag('statistics/sahel.csv')

In [ ]:
malawi = top5_admins_ag('statistics/Malawi_statistics_full.csv')

In [ ]:
malawi

## Top 5 Admins with Urban Trees
This function identifies the top five administrative districts with the highest tree cover estimates (average tree cover or total ha) on urban land cover types.  

Outstanding items to address:
- If ha of tree cover is the metric, should be weighted

In [ ]:
def top5_admins_urb(region):
    
    '''
    Filters dataframe to the ESA urban land cover class. 
    
    Calculates the total ha tree cover per admin or the comb avg  % tree cover per admin,
    then sorts values in descending order to get top 5 admins.
    
    '''
    
    df = pd.read_csv(f'/Volumes/Jessicas_Life/regional_stats/{region}.csv')
    
    # filter to only urban land cover class
    df = df[df.esa_id == 190.0]
    
    # get total (sum) ha tree cover in urban class for each admin
    df_ha = df.groupby(by=['country', 'admin']).sum().reset_index() 
    df_ha = df_ha[['country', 'admin', 'esa_sampled_ha', 'esa_total_ha','tof_ha']] 
    
    # add column with % of land cover class that was sampled
    df_ha['esa_perc_samp'] = round((df_ha.esa_sampled_ha / df_ha.esa_total_ha) * 100, 2)

    # now use original dataframe to get average tree cover in urban class for each admin
    df_avg = df.groupby(by=['country','admin']).mean().reset_index()
    df_avg = df_avg[['country', 'admin', 'tof_mean']]
    
    # merge dataframes
    df_merged = pd.merge(df_ha, df_avg, how='inner', left_on=['country','admin'], right_on=['country','admin'])
    
    # sort and print top 5 urban areas
    top_5_ha = df_merged.sort_values(['tof_ha', 'esa_perc_samp'], ascending=(False, False))[:5]
    top_5_ha = top_5_ha.reset_index().drop('index', 1)
    top_5_avg = df_merged.sort_values(['tof_mean', 'esa_perc_samp'], ascending=(False, False))[:5]
    top_5_avg = top_5_avg.reset_index().drop('index', 1)

    
    print('Top 5 Cities with Trees (total hectares tree cover)')
    for i in top_5_ha.index:
        print(f'{top_5_ha.esa_perc_samp[i]}% of urban areas were sampled in {top_5_ha.admin[i]}, ' \
              f'{top_5_ha.country[i]} with {top_5_ha.tof_ha[i]} ha of tree cover >0%.')
    
    print(' ')
    print('Top 5 Cities with Trees (average % tree cover)')
    for i in top_5_avg.index:
        print(f'{top_5_avg.esa_perc_samp[i]}% of urban areas were sampled in {top_5_avg.admin[i]}, ' \
              f'{top_5_avg.country[i]} with {round(top_5_avg.tof_mean[i], 0)}% average tree cover.')
        
    # display top 5 ha
#     num1 = rs.open(f'{top_5_ha.country[0]}/resampled_rasters/tof/{top_5_ha.admin[0]}.tif').read(1)
#     num2 = rs.open(f'{top_5_ha.country[1]}/resampled_rasters/tof/{top_5_ha.admin[1]}.tif').read(1)
    
#     plt.figure(figsize=(30,20))

#     plt.subplot(1,2,1)
#     plt.imshow(num1, cmap='Greens', vmin=0, vmax=100)
#     plt.title(f'#1: {top_5_ha.admin[0]}, {top_5_ha.country[0]} (ha tree cover)')

#     plt.subplot(1,2,2)
#     plt.imshow(num2, cmap='Greens', vmin=0, vmax=100)
#     plt.title(f'#2: {top_5_ha.admin[1]}, {top_5_ha.country[1]} (ha tree cover)')

    # figure out where in that admin has the highest tree cover by lat/lon coordinates?
    # long term could scale down to admin 2 level in these areas?

    return None

In [ ]:
top5_admins_urb('central_am')

In [ ]:
# new version for multi scale output

def urban_cover_admin(filepath):
    
    '''
    Filters dataframe to the following ESA urban land cover types:
    
    Urban
    
    Calculates the total ha tree cover per admin AND the avg % tree cover per admin.
    Merges the dataframes to include % of land cover class sampled and then builds
    two separate dataframes of top 5 cities in the region with highest stats.
    
    '''
    
    df = pd.read_csv(filepath)
    
    # filter to only urban land cover
    df = df[df.esa_id == 190]

    # get total (sum) ha tree cover in ag for each admin (drop ha <10% cover)
    df_ha = df.drop(df[df.tree_cover_class == '0-9'].index)
    df_ha = df.groupby(by=['country', 'admin']).sum().reset_index() 
    df_ha = df_ha[['country', 'admin', 'esa_sampled_ha', 'esa_total_ha','tof_ha']] 
    
    # add column with % of land cover class that was sampled
    df_ha['esa_perc_samp'] = round((df_ha.esa_sampled_ha / df_ha.esa_total_ha) * 100, 2)

    # now use original dataframe to get average tree cover in ag for each admin
    df_avg = df.groupby(by=['country','admin']).mean().reset_index()
    df_avg = df_avg[['country', 'admin', 'tof_mean']]
    
    # merge dataframes
    df_merged = pd.merge(df_ha, df_avg, how='inner', left_on=['country','admin'], right_on=['country','admin'])
    
    # sort and print top 5 ag areas
    top_5_ha = df_merged.sort_values(['tof_ha', 'esa_perc_samp'], ascending=(False, False))[:5]
    top_5_avg = df_merged.sort_values(['tof_mean', 'esa_perc_samp'], ascending=(False, False))[:5]
    
    print('Top 5 Admins with Urban Tree Cover (total hectares tree cover)')
    for i in top_5_ha.index:
        print(f'Urban areas in {top_5_ha.admin[i]}, {top_5_ha.country[i]} have {top_5_ha.tof_ha[i]} ' \
              f'ha of tree cover >10%. ({top_5_ha.esa_perc_samp[i]}% of urban areas were sampled)')
    
    print(' ')
    print('Top 5 Admins with Urban Tree Cover (average % tree cover)')
    for i in top_5_avg.index:
        print(f'Urban Areas in {top_5_avg.admin[i]}, {top_5_avg.country[i]} have {round(top_5_avg.tof_mean[i], 0)}% ' \
              f'average tree cover. ({top_5_avg.esa_perc_samp[i]}% of urban areas were sampled)')
    
    return None

In [ ]:
urban_cover_admin('statistics/El Salvador_statistics_full.csv')

## Tree Cover Distribution: Regional Comparisons
This grouped barchart compares tree cover distribution in 10% canopy cover thresholds across four regions: Central America, East Africa, West Africa, Sahel). (In the future could consider a facet grid that illustrates the distribution in a barchart per country)

In [ ]:
def tree_cover_distribution(figsize):
    
    # later will be updated to compare across 4 regions
    central_am = pd.read_csv('statistics/central_america_full.csv')
    #east_afr = pd.read_csv('regional_stats/east_afr.csv')
    west_afr = pd.read_csv('statistics/west_africa_full.csv')
    sahel = pd.read_csv('statistics/sahel.csv')

    central_am = central_am.groupby(by=['tree_cover_class']).sum().reset_index() 
    central_am = central_am[['tree_cover_class', 'tof_ha']]

    west_afr = west_afr.groupby(by=['tree_cover_class']).sum().reset_index() 
    west_afr = west_afr[['tree_cover_class', 'tof_ha']]
    
    sahel = sahel.groupby(by=['tree_cover_class']).sum().reset_index() 
    sahel = sahel[['tree_cover_class', 'tof_ha']]

    labels = list(central_am.tree_cover_class)
    x = np.arange(len(labels))  # the label locations
    width = 0.35  

    plt.figure(figsize=figsize)
    plt.bar(x - width/3, central_am.tof_ha, width, label='Central America')
    plt.bar(x + width/3, west_afr.tof_ha, width, label='West Africa')
    plt.bar(x + width/3, sahel.tof_ha, width, label='Sahel')

    plt.title('Tree Cover Distribution by Region')
    plt.xlabel('Thresholds (%)')
    plt.ylabel('Total Ha')
    plt.xticks(x, labels=labels)
    plt.ticklabel_format(style='plain', axis='y')
    plt.grid(axis='y', linestyle='-', linewidth=.2)
    plt.legend();
    
    return None

## Brazil

In [ ]:
# For large states, use admin2 boundaries
brazil = gpd.read_file('admin_boundaries/Brazil_shapefile/gadm41_BRA_2.shp')
para = brazil[brazil.NAME_1 == 'Pará']

# drop existing NAME_1 column and rename NAME_2
para = para.drop(columns='NAME_1')
para = para.rename(columns={"NAME_2": "NAME_1"})
para.to_file(f'admin_boundaries/Para_adminboundaries.geojson', driver='GeoJSON')

In [ ]:
# amazonas, bahia, mato grosso
brazil = gpd.read_file('admin_boundaries/Brazil_shapefile/gadm41_BRA_2.shp')
para = brazil[brazil.NAME_1 == 'Bahia']

# drop existing NAME_1 column and rename NAME_2
para = para.drop(columns='NAME_1')
para = para.rename(columns={"NAME_2": "NAME_1"})
para

In [ ]:
para.to_file(f'admin_boundaries/Bahia_adminboundaries.geojson', driver='GeoJSON')

In [ ]:
saudi = gpd.read_file('admin_boundaries/gadm41_SAU_shp/gadm41_SAU_2.shp')
saudi = saudi.drop(columns='NAME_1')
saudi = saudi.rename(columns={"NAME_2": "NAME_1"})
saudi.to_file(f'admin_boundaries/Saudi Arabia_adminboundaries.geojson', driver='GeoJSON')

## Caribbean

In [ ]:
## Create caribbean geojson admin 0
car0 = glob.glob('admin_boundaries/car_admin0/*.shp')
car1 = glob.glob('admin_boundaries/car_admin1/*.shp')

admin0_car = gpd.GeoDataFrame(pd.concat([gpd.read_file(shp) for shp in car0], ignore_index=True), 
                              crs=gpd.read_file(car0[0]).crs)

admin1_car = gpd.GeoDataFrame(pd.concat([gpd.read_file(shp) for shp in car1], ignore_index=True), 
                              crs=gpd.read_file(car1[0]).crs)


In [ ]:
admin0_car['NAME_1'] = admin0_car['COUNTRY']
admin0_car

In [ ]:
# combine
car_comb = gpd.GeoDataFrame(pd.concat([admin1_car, admin0_car], axis=0, ignore_index=True))

# save the whole thing as a geojson
car_comb.to_file(f'admin_boundaries/Caribbean_adminboundaries.geojson', driver='GeoJSON')

In [ ]:
# replace value of pandas column where admin is creating issues
shape = gpd.read_file('admin_boundaries/Caribbean_adminboundaries.geojson')

In [ ]:
shape.loc[shape['NAME_1'] == 'Mayaro/Rio Claro', 'NAME_1'] = 'Mayaro Rio Claro'

In [ ]:
# Tunapuna/Piarco
shape.loc[shape['NAME_1'] == 'Tunapuna/Piarco', 'NAME_1'] = 'Tunapuna Piarco'

In [ ]:
# confirm no others
set(list(shape.NAME_1))

In [ ]:
shape.to_file(f'admin_boundaries/Caribbean_adminboundaries.geojson', driver='GeoJSON')

In [ ]:
## Indonesia
Java (Indonesia)
Celebes (Indonesia)
Kalimantan (Indonesia)
Sumatra (Indonesia)
Papua (Indonesia)

In [ ]:
shapefile = f'/Users/jessica.ertel/Downloads/gadm41_IDN_shp/gadm41_IDN_1.shp'
new_shp = gpd.read_file(shapefile)
set(list(new_shp.NAME_1))

In [ ]:
new_shp.to_file(f'admin_boundaries/Indonesia_adminboundaries.geojson', driver='GeoJSON')

## Central America
### Average agricultural tree cover per admin

In [ ]:
### create central america admin 0 shapefile
shapefiles_ca = glob.glob('visualize/central_am_admin0/*.shp')
admin0_ca = gpd.GeoDataFrame(pd.concat([gpd.read_file(shp) for shp in shapefiles_ca], ignore_index=True), 
                              crs=gpd.read_file(shapefiles_ca[0]).crs)

In [ ]:
admin0_ca

In [ ]:
# create central america admin 1 (comb) shapefile
countries = ['El Salvador', 'Belize', 'Nicaragua', 'Guatemala', 'Honduras', 'Costa Rica', 'Panama']
admin1_ca = []
for country in countries:
    path = f'visualize/{country}_adminboundaries.geojson'
    admin1_ca.append(path)

gdf_ca = gpd.GeoDataFrame(pd.concat([gpd.read_file(shp) for shp in admin1_ca], ignore_index=True), 
                       crs=gpd.read_file(admin1_ca[0]).crs)
gdf_ca

In [ ]:
## now prepare the statistics csv
ag_ids = [10.0, 11.0, 12.0, 20.0, 30.0, 40.0]

ca_stats = pd.read_csv('statistics/central_america_full.csv')
ca_stats = ca_stats[['country', 'admin', 'esa_id', 'esa_class', 'tof_mean']]
ca_stats = ca_stats[ca_stats.esa_id.isin(ag_ids)]
ca_stats = ca_stats.groupby(by=['country','admin','esa_class']).mean().reset_index() #remove?

# now need to combine the ag classes and get one mean for the admin
ca_stats.drop(columns='esa_id', inplace=True)
ca_stats = ca_stats.groupby(by=['country','admin']).mean().reset_index()

# merge shapefile with statistics
comb_ca = gdf_ca.merge(ca_stats, left_on='NAME_1', right_on='admin')
comb_ca

In [ ]:
comb_ca[comb_ca.NAME_0 == 'Guatemala']

In [ ]:
bin_labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']
comb_ca['binned'] = pd.cut(comb_ca['tof_mean'],
                            bins=10,
                            labels=bin_labels)

ax = comb_ca.plot(column='binned', 
         cmap='Greens', 
         edgecolor='k',
         figsize=(20,20),
         linewidth=.3,
         legend=True)
#          vmin=0, 
#          vmax=100,
#          legend=True
#          #legend_kwds={'shrink': 0.3})

admin0_ca.plot(ax=ax,
            color='none',
            edgecolor='black',
            figsize=(20,20),
            linewidth=1.0)
plt.axis('off');

## Sahel

### Average agricultural tree cover per admin 

In [ ]:
### create sahel admin 0 shapefile
shapefiles = glob.glob('visualize/sahel_admin0/*.shp')
admin0 = gpd.GeoDataFrame(pd.concat([gpd.read_file(shp) for shp in shapefiles], ignore_index=True), 
                          crs=gpd.read_file(shapefiles[0]).crs)

In [ ]:
# create sahel admin 1 shapefile
countries = ['Burkina Faso', 'Cameroon', 'Chad', 'Ethiopia', 'Mali', 'Mauritania', 'Niger', 'Senegal']
countries2 = ['Central African Republic', 'Eritrea', 'Nigeria', 'South Sudan', 'Sudan']
shapefiles, shapefiles2 = [], []

In [ ]:
for country in countries:
    path = f'visualize/{country}_adminboundaries.geojson'
    shapefiles.append(path)
    
for country in countries2:
    path = f'visualize/{country}_adminboundaries.geojson'
    shapefiles2.append(path)

In [ ]:
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(shp) for shp in shapefiles], ignore_index=True), 
                       crs=gpd.read_file(shapefiles[0]).crs)

In [ ]:
gdf2 = gpd.GeoDataFrame(pd.concat([gpd.read_file(shp) for shp in shapefiles2], ignore_index=True), 
                       crs=gpd.read_file(shapefiles2[0]).crs)

In [ ]:
# combine gdf and gdf2 by updating gdf2
# drop ISO_1 column
# change ID_0 to GID_0 and ID_1 to GID_1
# change COUNTRY to NAME_0
gdf2.drop(columns='ISO_1', inplace=True)
gdf2.rename(columns={"ID_0": "GID_0", "COUNTRY": "NAME_0", "ID_1": "GID_1"}, inplace=True)

In [ ]:
final_gdf = pd.concat([gdf, gdf2], ignore_index=True)
final_gdf.shape

In [ ]:
# filter stats to only show tof_mean per land cover, per admin

#     Legend for ESA land cover categories:
#         10: 'Cropland, rainfed',
#         11: 'Cropland, rainfed',
#         12: 'Cropland, rainfed',
#         20: 'Cropland, irrigated or post-flooding',
#         30: 'Mosaic cropland / natural vegetation',
#         40: 'Mosaic natural vegetation / cropland',

ag_ids = [10.0, 11.0, 12.0, 20.0, 30.0, 40.0]

# when turning this into a function, takes input of which esa_class to visualize
sahel_stats = pd.read_csv('statistics/sahel.csv')
sahel_stats = sahel_stats[['country', 'admin', 'esa_id', 'esa_class', 'tof_mean']]
sahel_stats = sahel_stats[sahel_stats.esa_id.isin(ag_ids)]
sahel_stats = sahel_stats.groupby(by=['country','admin','esa_class']).mean().reset_index() #remove?

# now need to combine the ag classes and get one mean for the admin
sahel_stats.drop(columns='esa_id', inplace=True)
sahel_stats = sahel_stats.groupby(by=['country','admin']).mean().reset_index()

# merge shapefile with tree cover data
comb = final_gdf.merge(sahel_stats, left_on='NAME_1', right_on='admin')

In [ ]:
bin_labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']
comb['binned'] = pd.cut(comb['tof_mean'],
                        bins=10,
                        labels=bin_labels)

In [ ]:
ax = comb.plot(column='binned', 
         cmap='Greens', 
         edgecolor='k',
         figsize=(20,20),
         linewidth=.3,
         legend=False)
#          vmin=0, 
#          vmax=100,
#          legend=True
#          #legend_kwds={'shrink': 0.3})

admin0.plot(ax=ax,
            color='none',
            edgecolor='black',
            figsize=(20,20),
            linewidth=1.0)

#plt.title(f'Mean % tree cover on farms (per admin 1 boundary)')
plt.axis('off');

# ax = moz_admin.plot(color='none', edgecolor='black', linewidths=1.5)
# moz_admin_district.plot(ax=ax, color='none', edgecolor='grey', linewidths=0.5)

In [ ]:
#  input of which esa_class to visualize

def mean_tree_cover_farms(country):
    
    shapefile = gpd.read_file(f'visualize/{country}_adminboundaries.geojson')
    stats = pd.read_csv(f'statistics/{country}_statistics_full.csv')

    ag_ids = [10.0, 11.0, 12.0, 20.0, 30.0, 40.0]

    stats = stats[['country', 'admin', 'esa_id', 'esa_class', 'tof_mean']]
    stats = stats[stats.esa_id.isin(ag_ids)]
    stats = stats.groupby(by=['admin','esa_class']).mean().reset_index()
    stats.drop(columns='esa_id', inplace=True)
    stats = stats.groupby(by=['admin']).mean().reset_index()
    new = shapefile.merge(stats, left_on='NAME_1', right_on='admin')
    
    bin_labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']
    new['binned'] = pd.cut(new['tof_mean'], bins=10, labels=bin_labels)
    
#     if bin_avg == True:
#         plot_col = 'binned'
#     else: 
#         plot_col = 'tof_mean'
    
    new.plot(column='binned', 
             cmap='Greens', 
             edgecolor='k',
             linewidth=.3,
             figsize=(10,10),
             legend=False)
    plt.title(f'Mean tree cover on Farms in {country}')
    plt.axis('off');
    return None

In [ ]:
mean_tree_cover_farms('Ethiopia')
mean_tree_cover_farms('Eritrea')
mean_tree_cover_farms('Mali')
mean_tree_cover_farms('Niger')

### Open vs Closed Canopy Forests %

In [ ]:
region_df = pd.read_csv('statistics/sahel.csv')
    
countries = list(set(region_df.country.values))

x_labels, bar1, bar2 = [],[],[]

for country in countries: 
    
    # filter to one country and only urban/ag land cover classes
    country_df = region_df[region_df.country == country]

    # get total tof ha per tree cover threshold
    country_df = country_df.groupby(by=['country', 'tree_cover_class']).sum().reset_index() 
    country_df = country_df[['country', 'tree_cover_class', 'tof_ha']]
    
    # calculate # ha <10% tree cover and >10% tree cover
    tc_closed_open = sum(country_df.tof_ha[1:]) # >10%
    tc_open = sum(country_df.tof_ha[1:4]) # 10-40%
    tc_closed = sum(country_df.tof_ha[4:]) #40-100%
    
    # create percentage labels
    total = tc_closed + tc_open
    tc_open_perc = tc_open/total*100
    tc_closed_perc = tc_closed/total*100

    x_labels.append(country)
    bar1.append(tc_closed_perc)
    bar2.append(tc_open_perc)
    
# convert to array in order to add data labels
bar1 = np.asarray(bar1)
bar2 = np.asarray(bar2)

# using list comprehension + map() + lambda
# Replace substring in list of strings
x_labels_new = list(map(lambda x: str.replace(x, 'Central African Republic','CAR'), x_labels))

# create the figure - closed on bottom
plt.figure(figsize=(16,6))
plt.bar(x_labels_new, bar1, color="goldenrod", label='Closed (40-100% cover)')  
plt.bar(x_labels_new, bar2, bottom=bar1, color="khaki", label='Open (10-40% cover)')    

# labels
for xpos, ypos, yval in zip(x_labels_new, bar1/2, bar1):
    plt.text(xpos, ypos, f'{round(yval)}%', ha="center", va="center")
for xpos, ypos, yval in zip(x_labels_new, bar1 + bar2/2, bar2):
    plt.text(xpos, ypos, f'{round(yval)}%', ha="center", va="center")

plt.xlabel(' ')
#plt.ylabel('tree cover (ha)')
plt.yticks([])
plt.ticklabel_format(style='plain', axis='y')
plt.title(f'Open vs. Closed Canopy Forests')
plt.legend(loc='upper right');

### Total ha of trees on Farms

In [ ]:
## Create a table that shows total hectares of trees on farms per country

df = pd.read_csv('statistics/sahel.csv')
    
# filter to only ag land cover classes
ag_ids = [10.0, 11.0, 12.0, 20.0, 30.0, 40.0]
df = df[df.esa_id.isin(ag_ids)]

# get total (sum) ha tree cover in ag for country (drop ha <10% cover)
df_ha = df.drop(df[df.tree_cover_class == '0-9'].index)
df_ha = df.groupby(by=['country']).sum().reset_index() 
df_ha = df_ha[['country', 'tof_ha']] 
df_ha['tof_ha_rnd'] = round(df_ha.tof_ha / 1000000, 2)
# countries = ['Mali', 'Ethiopia', 'Niger', 'Eritrea']
# df_ha = df_ha[df_ha.country.isin(countries)]

In [ ]:
# create a horizontal bar chart w/ data labels
fig, ax = plt.subplots(figsize=(8,8))
sns.barplot(x='tof_ha_rnd', y='country', data=df_ha, color="green")
ax.set(xlim=(0, 50), 
       ylabel='',
       xlabel="Tree Cover (Mha)",
       title="Tree Cover (>10%) on Farms")
sns.set(font_scale=1, style='whitegrid');

In [ ]:
df = pd.read_csv('statistics/sahel.csv')
    
# filter to only ag land cover classes
grass_ids = [130]
df = df[df.esa_id.isin(grass_ids)]

# get total (sum) ha tree cover in ag for country (drop ha <10% cover)
df_ha = df.drop(df[df.tree_cover_class == '0-9'].index)
df_ha = df.groupby(by=['country']).sum().reset_index() 
df_ha = df_ha[['country', 'tof_ha']] 
df_ha['tof_ha_rnd'] = round(df_ha.tof_ha / 1000000, 2)

In [ ]:
# create a horizontal bar chart w/ data labels
fig, ax = plt.subplots(figsize=(8,8))
sns.barplot(x='tof_ha_rnd', y='country', data=df_ha, color="goldenrod")
ax.set(xlim=(0, 30), 
       ylabel='',
       xlabel="Tree Cover (Mha)",
       title="")
sns.set(font_scale=1, style='whitegrid');

### GGW Intervention Zones

In [ ]:
df = pd.read_csv('statistics/sahel.csv')
    
# filter to only ag land cover classes
ag_ids = [10.0, 11.0, 12.0, 20.0, 30.0, 40.0]
df = df[df.esa_id.isin(ag_ids)]

# get total (sum) ha tree cover in ag for country (drop ha <10% cover)
df_ha = df.drop(df[df.tree_cover_class == '0-9'].index)
df_ha = df.groupby(by=['country']).sum().reset_index() 
df_ha = df_ha[['country', 'tof_ha']] 
df_ha['tof_ha_rnd'] = round(df_ha.tof_ha / 1000000, 2)
countries = ['Mali', 'Ethiopia', 'Niger', 'Eritrea']
df_ha = df_ha[df_ha.country.isin(countries)]

# total GGW intervention area per country
df_ha['ggw_ha'] = [12.4, 13.2, 44.4, 47.3]

# create a horizontal bar chart w/ data labels
fig, ax = plt.subplots(figsize=(5,5))

x = np.arange(len(df_ha))
width = .4

plt.barh(x, df_ha.tof_ha_rnd, width, color='green', label='Tree cover >10%')
plt.barh(x+width, df_ha.ggw_ha, width, color='honeydew', label='Intervention area')

# sns.barplot(x='tof_ha_rnd', y='country', data=df_ha, color="green")
# sns.barplot(x='ggw_ha', y='country', data=df_ha, color="green", saturation=.1)

ax.set(xlim=(0, 50), 
       ylabel='',
       yticks=x + width,
       yticklabels=df_ha.country,
       xlabel="Tree Cover (Mha)",
       title="Trees on Farms in GGW Intervention Zones")
ax.legend();

## Malawi FLR Strategy

In [ ]:
# create a h barchart that shows total hectares of ag trees >10% for each admin in Malawi
df = pd.read_csv('statistics/Malawi_statistics_full.csv')
    
# filter to only ag land cover classes
ag_ids = [10.0, 11.0, 12.0, 20.0, 30.0, 40.0]
df = df[df.esa_id.isin(ag_ids)]

# get total (sum) ha tree cover in ag for country (drop ha <10% cover)
df_ha = df.drop(df[df.tree_cover_class == '0-9'].index)
df_ha = df.groupby(by=['admin']).sum().reset_index() 
df_ha = df_ha[['admin', 'tof_ha']].sort_values('admin', ascending=False)


# create a horizontal bar chart w/ data labels
fig, ax = plt.subplots(figsize=(5,10))

width = .3

plt.barh(y=df_ha.admin, width=df_ha.tof_ha, color='green')

ax.set(ylabel='',
       xticks=np.arange(0, 900000, step=100000),
       xlabel="Tree Cover (ha)",
       title="Malawi District Tree Cover on Farms in 2020 ")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.grid(color='grey', linestyle='-', linewidth=.3, axis='x');

In [ ]:
plt.figure(figsize=(8,6))
ax = sns.barplot(x='admin', y='tof_mean', data=malawi, palette='Set2')
ax.set(title='Average tree cover on cropland per district',
       ylabel='Avg (%)',
       xlabel= ' ')
ax.set_xticklabels(ax.get_xticklabels(), rotation=70);